# Libraries

In [1]:
# pyspark
!pip install pyspark

# findspark
!pip install findspark

# jupyterlab
!pip install jupyterlab

# unicode
!pip install Unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=bdcec1a41b9f212e445ed5a7860b7896ccc8847d9bafa43d18893a46831578c3
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 15.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import time
import pandas as pd

In [3]:
import findspark
findspark.init()

import requests
import json
import unidecode
from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .master("local[1]") \
      .appName("SparkByExamples.com") \
      .getOrCreate()

#IBGE API

In [4]:
ibge = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios").json()

In [5]:
ibge

[{'id': 3502507,
  'nome': 'Aparecida',
  'microrregiao': {'id': 35051,
   'nome': 'Guaratinguetá',
   'mesorregiao': {'id': 3513,
    'nome': 'Vale do Paraíba Paulista',
    'UF': {'id': 35,
     'sigla': 'SP',
     'nome': 'São Paulo',
     'regiao': {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}}}},
  'regiao-imediata': {'id': 350052,
   'nome': 'Guaratinguetá',
   'regiao-intermediaria': {'id': 3511,
    'nome': 'São José dos Campos',
    'UF': {'id': 35,
     'sigla': 'SP',
     'nome': 'São Paulo',
     'regiao': {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}}}}},
 {'id': 3503158,
  'nome': 'Arapeí',
  'microrregiao': {'id': 35052,
   'nome': 'Bananal',
   'mesorregiao': {'id': 3513,
    'nome': 'Vale do Paraíba Paulista',
    'UF': {'id': 35,
     'sigla': 'SP',
     'nome': 'São Paulo',
     'regiao': {'id': 3, 'sigla': 'SE', 'nome': 'Sudeste'}}}},
  'regiao-imediata': {'id': 350053,
   'nome': 'Cruzeiro',
   'regiao-intermediaria': {'id': 3511,
    'nome': 'São José dos Campos',
   

In [1]:
df_12 = spark.read.json(ibge)

NameError: ignored

In [6]:
# fist try/way to save data from API IBGE in lists
id_city = []
name_city = []
name_micro_region = []
name_messo_region = []
Abbreviation_state = []
name_state = []
Abbreviation_region = []
name_region = []


for i in range(len(ibge)):
    id_city.append(ibge[i]['id'])
    name_city.append(ibge[i]['nome'])
    name_micro_region.append(ibge[i]['microrregiao']['nome'])
    name_messo_region.append(ibge[i]['microrregiao']['mesorregiao']['nome'])
    Abbreviation_state.append(ibge[i]['microrregiao']['mesorregiao']['UF']['sigla'])
    name_state.append(ibge[i]['microrregiao']['mesorregiao']['UF']['nome'])
    Abbreviation_region.append(ibge[i]['microrregiao']['mesorregiao']['UF']['regiao']['sigla'])
    name_region.append(ibge[i]['microrregiao']['mesorregiao']['UF']['regiao']['nome'])

In [7]:
# second way to save data from IBGE api in one list just
list_df_ibge = []

for i in range(len(ibge)):
    list_df_ibge.append([ibge[i]['id'],
                        ibge[i]['nome'],
                        ibge[i]['microrregiao']['nome'],
                        ibge[i]['microrregiao']['mesorregiao']['nome'],
                        ibge[i]['microrregiao']['mesorregiao']['UF']['sigla'],
                        ibge[i]['microrregiao']['mesorregiao']['UF']['nome'],
                        ibge[i]['microrregiao']['mesorregiao']['UF']['regiao']['sigla'],
                        ibge[i]['microrregiao']['mesorregiao']['UF']['regiao']['nome']])


In [8]:
list_df_ibge

[[3502507,
  'Aparecida',
  'Guaratinguetá',
  'Vale do Paraíba Paulista',
  'SP',
  'São Paulo',
  'SE',
  'Sudeste'],
 [3503158,
  'Arapeí',
  'Bananal',
  'Vale do Paraíba Paulista',
  'SP',
  'São Paulo',
  'SE',
  'Sudeste'],
 [3503505,
  'Areias',
  'Bananal',
  'Vale do Paraíba Paulista',
  'SP',
  'São Paulo',
  'SE',
  'Sudeste'],
 [3504909,
  'Bananal',
  'Bananal',
  'Vale do Paraíba Paulista',
  'SP',
  'São Paulo',
  'SE',
  'Sudeste'],
 [3508504,
  'Caçapava',
  'São José dos Campos',
  'Vale do Paraíba Paulista',
  'SP',
  'São Paulo',
  'SE',
  'Sudeste'],
 [3508603,
  'Cachoeira Paulista',
  'Guaratinguetá',
  'Vale do Paraíba Paulista',
  'SP',
  'São Paulo',
  'SE',
  'Sudeste'],
 [3509700,
  'Campos do Jordão',
  'Campos do Jordão',
  'Vale do Paraíba Paulista',
  'SP',
  'São Paulo',
  'SE',
  'Sudeste'],
 [3509957,
  'Canas',
  'Guaratinguetá',
  'Vale do Paraíba Paulista',
  'SP',
  'São Paulo',
  'SE',
  'Sudeste'],
 [3510500,
  'Caraguatatuba',
  'Caraguatatuba

In [9]:
#create dataframe with list from IBGE API
df_ibge = pd.DataFrame(list_df_ibge, columns=['id_city','name_city','name_micro_region','name_messo_region',
                                                 'Abbreviation_state','name_state','Abbreviation_region','name_region'])

In [10]:

df_ibge.name_city = df_ibge.name_city.str.normalize('NFKD').str.encode('ascii',errors='ignore').str.decode('utf-8')

In [11]:
df_ibge

,id_city,name_city,name_micro_region,name_messo_region,Abbreviation_state,name_state,Abbreviation_region,name_region
0,3502507,Aparecida,Guaratinguetá,Vale do Paraíba Paulista,SP,São Paulo,SE,Sudeste
1,3503158,Arapei,Bananal,Vale do Paraíba Paulista,SP,São Paulo,SE,Sudeste
2,3503505,Areias,Bananal,Vale do Paraíba Paulista,SP,São Paulo,SE,Sudeste
3,3504909,Bananal,Bananal,Vale do Paraíba Paulista,SP,São Paulo,SE,Sudeste
4,3508504,Cacapava,São José dos Campos,Vale do Paraíba Paulista,SP,São Paulo,SE,Sudeste
5,3508603,Cachoeira Paulista,Guaratinguetá,Vale do Paraíba Paulista,SP,São Paulo,SE,Sudeste
6,3509700,Campos do Jordao,Campos do Jordão,Vale do Paraíba Paulista,SP,São Paulo,SE,Sudeste
7,3509957,Canas,Guaratinguetá,Vale do Paraíba Paulista,SP,São Paulo,SE,Sudeste
8,3510500,Caraguatatuba,Caraguatatuba,Vale do Paraíba Paulista,SP,São Paulo,SE,Sudeste
9,3513405,Cruzeiro,Guaratinguetá,Vale do Paraíba Paulista,SP,São Paulo,SE,Sudeste


#Weather API

In [12]:
# #df without 3 cities - that we can't find in the weather api
df_ibge2 = df_ibge.drop(index=[24]).reset_index(drop=True)

In [13]:
# clear the city names
import unicodedata
df_ibge2.name_city = df_ibge2.name_city.str.normalize('NFKD').str.encode('ascii',errors='ignore').str.decode('utf-8')

In [14]:
df_ibge.name_city

0                   Aparecida
1                      Arapei
2                      Areias
3                     Bananal
4                    Cacapava
5          Cachoeira Paulista
6            Campos do Jordao
7                       Canas
8               Caraguatatuba
9                    Cruzeiro
10                      Cunha
11              Guaratingueta
12                    Igarata
13                   Ilhabela
14                    Jacarei
15                   Jambeiro
16                   Lagoinha
17                  Lavrinhas
18                     Lorena
19            Monteiro Lobato
20        Natividade da Serra
21                  Paraibuna
22            Pindamonhangaba
23                    Piquete
24                      Potim
25                     Queluz
26          Redencao da Serra
27                    Roseira
28               Santa Branca
29    Santo Antonio do Pinhal
30       Sao Bento do Sapucai
31       Sao Jose do Barreiro
32        Sao Jose dos Campos
33     Sao

In [15]:
#firts try
# # code to get data from weather api and create a dataframe
for name_city in df_ibge2.name_city	:
  http_weather = 'http://api.weatherapi.com/v1/forecast.json?key=eec58040ec21402cabe173206232203&q='+name_city+'&days=6&aqi=no&alerts=no'
  weather_request = requests.get(http_weather).json()
  list_weather_keys = []
  list_weather_value = []
  for key, value in weather_request['location'].items(): 
        list_weather_value.append(value)
        if name_city == df_ibge2.name_city[0]:
            list_weather_keys.append(key)
  for i in range(6):
      list_weather_value.append(weather_request['forecast']['forecastday'][i]['date'])
      if name_city == df_ibge2.name_city[0]:
          list_weather_keys.append('date'+str(i))
      for key,value in weather_request['forecast']['forecastday'][i]['day'].items():
              if name_city == df_ibge2.name_city[0]:
                  list_weather_keys.append(key+"day"+str(i))
                  df_weather = pd.DataFrame(columns = [list_weather_keys])
              if key == "condition":
                  list_weather_value.append(weather_request['forecast']['forecastday'][i]['day']['condition']['text'])
              else:
                  list_weather_value.append(value)   
  df_weather.loc[len(df_weather)] = list_weather_value    


  

In [16]:
# # code to get data from weather api and create a dataframe
# second try
list_weather_keys = []
for name_city in df_ibge2.name_city	:
  http_weather = 'http://api.weatherapi.com/v1/forecast.json?key=eec58040ec21402cabe173206232203&q='+name_city+'&days=6&aqi=no&alerts=no'
  weather_request = requests.get(http_weather).json()
  for i in range(6):
      list_weather_value = []
      for key, value in weather_request['location'].items(): 
            list_weather_value.append(value)
            if name_city == df_ibge2.name_city[0] and i == 0:
                list_weather_keys.append(key)
      list_weather_value.append(weather_request['forecast']['forecastday'][i]['date'])
      if name_city == df_ibge2.name_city[0] and i == 0:
          list_weather_keys.append('date')
      for key,value in weather_request['forecast']['forecastday'][i]['day'].items():
              if name_city == df_ibge2.name_city[0] and i == 0:
                  list_weather_keys.append(key)
                  df_weather = pd.DataFrame(columns = [list_weather_keys])
              if key == "condition":
                  list_weather_value.append(weather_request['forecast']['forecastday'][i]['day']['condition']['text'])
              else:
                  list_weather_value.append(value)   
      df_weather.loc[len(df_weather)] = list_weather_value 

In [17]:
df_weather

,name,region,country,lat,lon,tz_id,localtime_epoch,localtime,date,maxtemp_c,...,totalsnow_cm,avgvis_km,avgvis_miles,avghumidity,daily_will_it_rain,daily_chance_of_rain,daily_will_it_snow,daily_chance_of_snow,condition,uv
0,Aparecida,Sao Paulo,Brazil,-22.83,-45.23,America/Sao_Paulo,1679962679,2023-03-27 21:17,2023-03-27,26.5,...,0.0,7.3,4.0,92.0,1,88,0,0,Moderate rain,5.0
1,Aparecida,Sao Paulo,Brazil,-22.83,-45.23,America/Sao_Paulo,1679962679,2023-03-27 21:17,2023-03-28,27.8,...,0.0,6.8,4.0,90.0,1,88,0,0,Heavy rain,5.0
2,Aparecida,Sao Paulo,Brazil,-22.83,-45.23,America/Sao_Paulo,1679962679,2023-03-27 21:17,2023-03-29,29.4,...,0.0,7.0,4.0,87.0,1,89,0,0,Moderate rain,5.0
3,Aparecida,Sao Paulo,Brazil,-22.83,-45.23,America/Sao_Paulo,1679962679,2023-03-27 21:17,2023-03-30,32.0,...,0.0,7.2,4.0,85.0,1,89,0,0,Moderate rain,5.0
4,Aparecida,Sao Paulo,Brazil,-22.83,-45.23,America/Sao_Paulo,1679962679,2023-03-27 21:17,2023-03-31,28.5,...,0.0,8.0,4.0,86.0,1,88,0,0,Moderate rain,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,Ubatuba,Sao Paulo,Brazil,-23.43,-45.07,America/Sao_Paulo,1679962696,2023-03-27 21:18,2023-03-28,26.7,...,0.0,10.0,6.0,87.0,1,91,0,0,Patchy rain possible,5.0
224,Ubatuba,Sao Paulo,Brazil,-23.43,-45.07,America/Sao_Paulo,1679962696,2023-03-27 21:18,2023-03-29,29.7,...,0.0,5.9,3.0,89.0,1,89,0,0,Moderate rain,5.0
225,Ubatuba,Sao Paulo,Brazil,-23.43,-45.07,America/Sao_Paulo,1679962696,2023-03-27 21:18,2023-03-30,33.0,...,0.0,7.8,4.0,85.0,1,84,0,0,Moderate rain,5.0
226,Ubatuba,Sao Paulo,Brazil,-23.43,-45.07,America/Sao_Paulo,1679962696,2023-03-27 21:18,2023-03-31,31.5,...,0.0,9.7,6.0,86.0,1,80,0,0,Moderate rain,5.0


In [18]:
df_weather['daily_will_it_rain'] = df_weather['daily_will_it_rain'].replace({0: 'não', 1: 'sim'})  

In [19]:
df_weather

,name,region,country,lat,lon,tz_id,localtime_epoch,localtime,date,maxtemp_c,...,totalsnow_cm,avgvis_km,avgvis_miles,avghumidity,daily_will_it_rain,daily_chance_of_rain,daily_will_it_snow,daily_chance_of_snow,condition,uv
0,Aparecida,Sao Paulo,Brazil,-22.83,-45.23,America/Sao_Paulo,1679962679,2023-03-27 21:17,2023-03-27,26.5,...,0.0,7.3,4.0,92.0,sim,88,0,0,Moderate rain,5.0
1,Aparecida,Sao Paulo,Brazil,-22.83,-45.23,America/Sao_Paulo,1679962679,2023-03-27 21:17,2023-03-28,27.8,...,0.0,6.8,4.0,90.0,sim,88,0,0,Heavy rain,5.0
2,Aparecida,Sao Paulo,Brazil,-22.83,-45.23,America/Sao_Paulo,1679962679,2023-03-27 21:17,2023-03-29,29.4,...,0.0,7.0,4.0,87.0,sim,89,0,0,Moderate rain,5.0
3,Aparecida,Sao Paulo,Brazil,-22.83,-45.23,America/Sao_Paulo,1679962679,2023-03-27 21:17,2023-03-30,32.0,...,0.0,7.2,4.0,85.0,sim,89,0,0,Moderate rain,5.0
4,Aparecida,Sao Paulo,Brazil,-22.83,-45.23,America/Sao_Paulo,1679962679,2023-03-27 21:17,2023-03-31,28.5,...,0.0,8.0,4.0,86.0,sim,88,0,0,Moderate rain,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,Ubatuba,Sao Paulo,Brazil,-23.43,-45.07,America/Sao_Paulo,1679962696,2023-03-27 21:18,2023-03-28,26.7,...,0.0,10.0,6.0,87.0,sim,91,0,0,Patchy rain possible,5.0
224,Ubatuba,Sao Paulo,Brazil,-23.43,-45.07,America/Sao_Paulo,1679962696,2023-03-27 21:18,2023-03-29,29.7,...,0.0,5.9,3.0,89.0,sim,89,0,0,Moderate rain,5.0
225,Ubatuba,Sao Paulo,Brazil,-23.43,-45.07,America/Sao_Paulo,1679962696,2023-03-27 21:18,2023-03-30,33.0,...,0.0,7.8,4.0,85.0,sim,84,0,0,Moderate rain,5.0
226,Ubatuba,Sao Paulo,Brazil,-23.43,-45.07,America/Sao_Paulo,1679962696,2023-03-27 21:18,2023-03-31,31.5,...,0.0,9.7,6.0,86.0,sim,80,0,0,Moderate rain,5.0


# Spark

In [20]:
# create a spark df
df_spark_weather = spark.createDataFrame(df_weather,list_weather_keys)
df_spark_weather.show()

+---------+---------+-------+------+------+-----------------+---------------+----------------+----------+---------+---------+---------+---------+---------+---------+-----------+-----------+--------------+--------------+------------+---------+------------+-----------+------------------+--------------------+------------------+--------------------+--------------------+---+
|     name|   region|country|   lat|   lon|            tz_id|localtime_epoch|       localtime|      date|maxtemp_c|maxtemp_f|mintemp_c|mintemp_f|avgtemp_c|avgtemp_f|maxwind_mph|maxwind_kph|totalprecip_mm|totalprecip_in|totalsnow_cm|avgvis_km|avgvis_miles|avghumidity|daily_will_it_rain|daily_chance_of_rain|daily_will_it_snow|daily_chance_of_snow|           condition| uv|
+---------+---------+-------+------+------+-----------------+---------------+----------------+----------+---------+---------+---------+---------+---------+---------+-----------+-----------+--------------+--------------+------------+---------+------------

In [21]:
df_spark_ibge = spark.createDataFrame(df_ibge2)
df_spark_ibge.show()

+-------+------------------+--------------------+--------------------+------------------+----------+-------------------+-----------+
|id_city|         name_city|   name_micro_region|   name_messo_region|Abbreviation_state|name_state|Abbreviation_region|name_region|
+-------+------------------+--------------------+--------------------+------------------+----------+-------------------+-----------+
|3502507|         Aparecida|       Guaratinguetá|Vale do Paraíba P...|                SP| São Paulo|                 SE|    Sudeste|
|3503158|            Arapei|             Bananal|Vale do Paraíba P...|                SP| São Paulo|                 SE|    Sudeste|
|3503505|            Areias|             Bananal|Vale do Paraíba P...|                SP| São Paulo|                 SE|    Sudeste|
|3504909|           Bananal|             Bananal|Vale do Paraíba P...|                SP| São Paulo|                 SE|    Sudeste|
|3508504|          Cacapava| São José dos Campos|Vale do Paraíba P...

In [22]:
df_spark_weather.createOrReplaceTempView('forecast')
df_spark_ibge.createOrReplaceTempView('cities')

In [23]:
spark.sql("select id_city, name_city from cities").show()

+-------+------------------+
|id_city|         name_city|
+-------+------------------+
|3502507|         Aparecida|
|3503158|            Arapei|
|3503505|            Areias|
|3504909|           Bananal|
|3508504|          Cacapava|
|3508603|Cachoeira Paulista|
|3509700|  Campos do Jordao|
|3509957|             Canas|
|3510500|     Caraguatatuba|
|3513405|          Cruzeiro|
|3513603|             Cunha|
|3518404|     Guaratingueta|
|3520202|           Igarata|
|3520400|          Ilhabela|
|3524402|           Jacarei|
|3524907|          Jambeiro|
|3526308|          Lagoinha|
|3526605|         Lavrinhas|
|3527207|            Lorena|
|3531704|   Monteiro Lobato|
+-------+------------------+
only showing top 20 rows



In [24]:
spark.sql("select name, country from forecast").show()

+---------+-------+
|     name|country|
+---------+-------+
|Aparecida| Brazil|
|Aparecida| Brazil|
|Aparecida| Brazil|
|Aparecida| Brazil|
|Aparecida| Brazil|
|Aparecida| Brazil|
|   Arapei| Brazil|
|   Arapei| Brazil|
|   Arapei| Brazil|
|   Arapei| Brazil|
|   Arapei| Brazil|
|   Arapei| Brazil|
|   Areias| Brazil|
|   Areias| Brazil|
|   Areias| Brazil|
|   Areias| Brazil|
|   Areias| Brazil|
|   Areias| Brazil|
|  Bananal| Brazil|
|  Bananal| Brazil|
+---------+-------+
only showing top 20 rows



In [25]:
spark.sql("select forecast.name, cities.name_state, forecast.region ,SUM(forecast.country) from forecast join cities on forecast.name = cities.name_city GROUP BY forecast.name, cities.name_state, forecast.region ").show()

+------------------+----------+------------+------------+
|              name|name_state|      region|sum(country)|
+------------------+----------+------------+------------+
|         Aparecida| São Paulo|   Sao Paulo|        null|
|            Arapei| São Paulo|   Sao Paulo|        null|
|            Areias| São Paulo|        Para|        null|
|           Bananal| São Paulo|    Maranhao|        null|
|          Cacapava| São Paulo|   Sao Paulo|        null|
|Cachoeira Paulista| São Paulo|   Sao Paulo|        null|
|             Canas| São Paulo|  Guanacaste|        null|
|     Caraguatatuba| São Paulo|   Sao Paulo|        null|
|             Cunha| São Paulo|   Sao Paulo|        null|
|     Guaratingueta| São Paulo|   Sao Paulo|        null|
|           Igarata| São Paulo|   Sao Paulo|        null|
|          Ilhabela| São Paulo|   Sao Paulo|        null|
|           Jacarei| São Paulo|   Sao Paulo|        null|
|          Jambeiro| São Paulo|Minas Gerais|        null|
|          Lag

In [26]:
#TABELA 1
spark.sql('SELECT f.name AS Cidade, \
                  c.id_city AS CodigoDaCidade, \
                  f.localtime AS DiaHora_Atual, \
                  f.date AS Data, \
                  c.name_region AS Regiao, \
                  c.name_state AS Estado, \
                  f.country AS Pais, \
                  f.lat AS Latitude, \
                  f.lon AS Longigute, \
                  f.maxtemp_c AS TemperaturaMaxima, \
                  f.mintemp_c AS TemperaturaMinima, \
                  f.avgtemp_c AS TemperaturaMedia, \
                  f.daily_will_it_rain AS VaiChover, \
                  f.daily_chance_of_rain AS ChanceDeChuva, \
                  f.condition AS CondicaoDoTempo, \
                  f.maxwind_mph AS VelocidadeMaximaDoVento \
          FROM forecast as f \
          JOIN cities as c \
          ON f.name = c.name_city').show()















+---------+--------------+----------------+----------+-------+---------+------+--------+---------+-----------------+-----------------+----------------+---------+-------------+--------------------+-----------------------+
|   Cidade|CodigoDaCidade|   DiaHora_Atual|      Data| Regiao|   Estado|  Pais|Latitude|Longigute|TemperaturaMaxima|TemperaturaMinima|TemperaturaMedia|VaiChover|ChanceDeChuva|     CondicaoDoTempo|VelocidadeMaximaDoVento|
+---------+--------------+----------------+----------+-------+---------+------+--------+---------+-----------------+-----------------+----------------+---------+-------------+--------------------+-----------------------+
|Aparecida|       3502507|2023-03-27 21:17|2023-03-27|Sudeste|São Paulo|Brazil|  -22.83|   -45.23|             26.5|             14.5|            20.5|      sim|           88|       Moderate rain|                    6.7|
|Aparecida|       3502507|2023-03-27 21:17|2023-03-28|Sudeste|São Paulo|Brazil|  -22.83|   -45.23|             27.8|

In [27]:
spark.sql("SELECT f.name AS Cidade, \
                  f.region AS Estado, \
                  COUNT(f.daily_will_it_rain) AS QtdDiasVaiChover, \
                  COUNT(f.name) AS TotalDiasMapeados\
          FROM forecast as f \
          JOIN cities as c \
          ON f.name = c.name_city \
          WHERE f.daily_will_it_rain = 'sim' \
          GROUP BY 1,2").show(40)

+------------------+----------+----------------+-----------------+
|            Cidade|    Estado|QtdDiasVaiChover|TotalDiasMapeados|
+------------------+----------+----------------+-----------------+
|         Aparecida| Sao Paulo|               6|                6|
|            Arapei| Sao Paulo|               6|                6|
|            Areias|      Para|               6|                6|
|           Bananal|  Maranhao|               6|                6|
|          Cacapava| Sao Paulo|               6|                6|
|Cachoeira Paulista| Sao Paulo|               6|                6|
|             Canas|Guanacaste|               1|                1|
|     Caraguatatuba| Sao Paulo|               6|                6|
|             Cunha| Sao Paulo|               6|                6|
|     Guaratingueta| Sao Paulo|               6|                6|
|           Igarata| Sao Paulo|               6|                6|
|          Ilhabela| Sao Paulo|               6|              

In [28]:
#TABELA 2
spark.sql("SELECT f.name AS Cidade, \
                  f.region AS Estado, \
                  COUNT(CASE WHEN f.daily_will_it_rain = 'sim' THEN 1 ELSE NULL END) AS QtdDiasVaiChover, \
                  COUNT(CASE WHEN f.daily_will_it_rain = 'não' THEN 1 ELSE NULL END) AS QtdDiasNaoVaiChover, \
                  COUNT(f.name) AS TotalDiasMapeados \
          FROM forecast as f \
          JOIN cities as c \
          ON f.name = c.name_city \
          GROUP BY 1,2").show(40)

+------------------+------------+----------------+-------------------+-----------------+
|            Cidade|      Estado|QtdDiasVaiChover|QtdDiasNaoVaiChover|TotalDiasMapeados|
+------------------+------------+----------------+-------------------+-----------------+
|         Aparecida|   Sao Paulo|               6|                  0|                6|
|            Arapei|   Sao Paulo|               6|                  0|                6|
|            Areias|        Para|               6|                  0|                6|
|           Bananal|    Maranhao|               6|                  0|                6|
|          Cacapava|   Sao Paulo|               6|                  0|                6|
|Cachoeira Paulista|   Sao Paulo|               6|                  0|                6|
|             Canas|  Guanacaste|               1|                  5|                6|
|     Caraguatatuba|   Sao Paulo|               6|                  0|                6|
|             Cunha| 

#Extra

In [29]:
ibge_teste_2 = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/estados/RS/distritos").json()

In [60]:
def ibge_cidades(uf,cidade):
  link_uf = 'https://servicodados.ibge.gov.br/api/v1/localidades/estados/'+uf+'/distritos'
  get_uf = requests.get(link_uf).json()
  for i in range(len(get_uf)):
    if cidade == get_uf[i]['nome']:
      # print('id =',get_uf[i]['id'],'- cidade:',get_uf[i]['nome'])
      id_cidade = i
  list_df_ibge = []
  list_df_ibge.append([get_uf[id_cidade]['municipio']['id'],
                      get_uf[id_cidade]['municipio']['nome'],
                      get_uf[id_cidade]['municipio']['microrregiao']['nome'],
                      get_uf[id_cidade]['municipio']['microrregiao']['mesorregiao']['nome'],
                      get_uf[id_cidade]['municipio']['microrregiao']['mesorregiao']['UF']['nome'],
                      get_uf[id_cidade]['municipio']['microrregiao']['mesorregiao']['UF']['sigla']])
  return list_df_ibge


In [62]:
cidade = 'Arvorezinha'
uf = 'rs'
ibge_cidades(uf,cidade)

[[4301404,
  'Arvorezinha',
  'Guaporé',
  'Nordeste Rio-grandense',
  'Rio Grande do Sul',
  'RS']]